In [ ]:
import sys
sys.path.append('../')
from __future__ import print_function
import pdb
from PIL import Image
import matplotlib.pyplot as plt
import sys
import random
import os
import argparse
import torch
import pandas as pd
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data.dataloader import DATA_ROOT
from data.transform import BaseTransform
from data.config import HOME
from PIL import Image
from ssd import *
import torch.utils.data as data
from data.dataloader import provider
from submission import *

%matplotlib inline

In [ ]:
cuda=True
if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')
visual_threshold = 0.6
device = torch.device("cuda:0" if cuda else "cpu")

trained_model_path = '../weights/model.pth'

In [ ]:
# load net
num_classes = len(CLASSES) + 1 # +1 background
net = build_ssd('test', 300, num_classes) # initialize SSD
net = torch.nn.DataParallel(net)

In [ ]:
# if not cuda:
#     net.load_state_dict(torch.load(trained_model_path,  map_location=lambda storage, loc: storage)['state'])
# else:
net.load_state_dict(torch.load(trained_model_path)['state_dict'])
net = net.to(device)
net.eval();

## Analyse the model on dataloader

In [ ]:
dataloader = provider(phase='val', batch_size=8)

In [ ]:
fnames, images, boxes = next(iter(dataloader))
images = images.to(device)
pred = net(images)
detections = pred.data
iter_range = iter(range(len(images)))

In [ ]:
for i, box in enumerate(boxes):
    print(i, box)

In [ ]:
for i in range(len(images)):
    print(i, detections[i, 1, :9, 0])

In [ ]:
'''Plot random test images with trained SSD's predictions'''
# iter_range = iter(range(len(images)))
# i = random.choice(list(range(len(images))))
i = next(iter_range)
# i=7
print(i, detections[i, 1, :5, 0])
fig, ax = plt.subplots(figsize=(10, 10))
for box in boxes[i]:
    box = [i* 300 for i in box]
    XMin, YMin, XMax, YMax = box[0], box[1], box[2], box[3]
    w = XMax - XMin + 1
    h = YMax - YMin + 1
    rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor='g', fill=None, linewidth=2)
    ax.add_patch(rect)

scale = torch.Tensor([300] * 4)
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

for idx in np.where(detections[i, 1, :, 0] >= 0.2)[0]:
    score = detections[i, 1, idx, 0]
    XMin, YMin, XMax, YMax = [x * 300 for x in detections[i, 1, idx, 1:]]
    w = XMax - XMin + 1
    h = YMax - YMin + 1
    rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor='r', fill=None, linewidth=2)
    ax.add_patch(rect)

ax.imshow(images.data[i][0] * 255, cmap="gist_gray")
plt.show();
# Red => prediction, Green=Ground Truth

## Analyse on test set

In [ ]:
# load data
root = os.path.join(HOME, "data/stage_1_test_images/")
sample_submission_path =  os.path.join(HOME, "data/stage_1_sample_submission.csv")
testset = TestDataset(
    root,
    sample_submission_path,
)
transform = BaseTransform(net.module.size, (104, 117, 123))
num_images = len(testset)


In [ ]:
iter_range = iter(range(num_images))

In [ ]:
i = next(iter_range)
img = testset.pull_image(i)
x = torch.Tensor(img).view(1, 3, 300, 300)
x = x.to(device)
y = net(x)      # forward pass
detections = y.data
print(i, detections[0, 1, :5, 0])
fig, ax = plt.subplots(figsize=(10, 10))
scale = torch.Tensor([300] * 4)
for idx in np.where(detections[0, 1, :, 0] >= 0.2)[0]:
    score = detections[0, 1, idx, 0]
    XMin, YMin, XMax, YMax = [x * 300 for x in detections[0, 1, idx, 1:]]
    w = XMax - XMin + 1
    h = YMax - YMin + 1
    rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor='r', fill=None, linewidth=2)
    ax.add_patch(rect)

ax.imshow((img.transpose(1,2,0) * 255).astype('int'), cmap="gist_gray")
plt.show();
# Red => prediction, Green=Ground Truth
